### Predicting Causal Estimates with Limited Supervision
In this notebook we:

- Utilize a graph neural network to predict the effect of a marketing campaign to the user's consumption.

- Compare our model with standard causal machine learning methods.


### Load the data

In [2]:
import torch

data = torch.load("../data/retail/processed/data.pt")[0]
data

HeteroData(
  (user, buys, product)={
    edge_index=[2, 14543339],
    treatment=[14543339],
  },
  user={
    x=[180653, 7],
    t=[180653],
    y=[180653],
  },
  products={ num_products=40542 }
)

### Loss and Evaluation Functions

In [3]:
import torch.nn.functional as F

def outcome_regression_loss_l1(t_true: torch.tensor,
                               y_treatment_pred: torch.tensor, 
                               y_control_pred: torch.tensor, 
                               y_true: torch.tensor) -> torch.tensor:
    """
    Compute mse for treatment and control output layers using treatment vector for masking out the counterfactual predictions
    """
    loss0 = torch.mean(((1. - t_true) * F.l1_loss(y_control_pred.squeeze(), y_true, reduction='none')) )
    loss1 = torch.mean((t_true *  F.l1_loss(y_treatment_pred.squeeze(), y_true, reduction='none') ))

    return (loss0 + loss1)/2

In [4]:
import numpy as np

def uplift_score(prediction: torch.tensor, 
                 treatment: torch.tensor, 
                 target: torch.tensor, 
                 rate=0.2) -> float:
    """
    From https://ods.ai/competitions/x5-retailhero-uplift-modeling/data
    Order the samples by the predicted uplift. 
    Calculate the average ground truth outcome of the top rate*100% of the treated and the control samples.
    Subtract the above to get the uplift. 
    """
    order = np.argsort(-prediction)
    treatment_n = int((treatment == 1).sum() * rate)
    treatment_p = target[order][treatment[order] == 1][:treatment_n].mean()

    control_n = int((treatment == 0).sum() * rate)
    control_p = target[order][treatment[order] == 0][:control_n].mean()
    score = treatment_p - control_p
    return score

### Define training parameters

In [5]:
import random

n_hidden = 16
lr = 0.01
l2_reg = 5e-4 
dropout = 0.2

no_layers = 1 
out_channels = 1
num_epochs = 300

seed = 0
k = 10
validation_fraction = 5
patience = 50

criterion = outcome_regression_loss_l1


torch.manual_seed(seed) 
np.random.seed(seed)
random.seed(seed)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

xp = torch.eye(data['products']['num_products']).to(device)
xu = data['user']['x'].to(device)
treatment = data['user']['t'].to(device)
outcome = data['user']['y'].to(device)


In [8]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=abs(k), shuffle=True, random_state=seed)

for train_indices, test_indices in kf.split(xu):
    test_indices, train_indices = train_indices, test_indices
    break 

val_indices = train_indices[:int(len(train_indices)/validation_fraction)]
train_indices = train_indices[int(len(train_indices)/validation_fraction):]

### Keep the graph before the treatment and ONLY the edges of the the train nodes (i.e. after the treatment)

In [9]:
mask = torch.isin(data['user','buys','product']['edge_index'][0, :], torch.tensor(train_indices) )
edge_index_up_current = data['user','buys','product']['edge_index'][ : , (~data['user','buys','product']['treatment']) | (mask) ]


#interaction to adjecency matrix
edge_index_up_current[1] = edge_index_up_current[1]+ xu.shape[0]

edge_index_up_current = torch.cat([edge_index_up_current,edge_index_up_current.flip(dims=[0])],dim=1).to(device)

In [10]:
edge_index = data['user','buys','product']['edge_index'][ : , (~data['user','buys','product']['treatment']) | (mask) ]
sparse_matrix_edge_index = torch.sparse_coo_tensor(
            edge_index,
            torch.ones(edge_index.shape[1]),
            (xu.shape[0], xp.shape[0]),
            dtype=torch.float
        )

In [11]:
def make_treatment_feature(x, train_indices, treatment):
    t_hat = torch.zeros(x.size(0), 2, dtype=torch.float)
    t_hat[train_indices,treatment.type(torch.LongTensor)[train_indices]]=1
    return t_hat

new_feature = make_treatment_feature(xu, train_indices, treatment)

In [12]:
product_treatment_matrix = torch.sparse.mm(sparse_matrix_edge_index.t(), new_feature.to_sparse())
user_treatment_matrix = torch.sparse.mm(sparse_matrix_edge_index, product_treatment_matrix)

/var/folders/nd/nn8th6r16cq6sytrzw1nw_9w0000gn/T/ipykernel_55945/3973650620.py:1: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:55.)
  product_treatment_matrix = torch.sparse.mm(sparse_matrix_edge_index.t(), new_feature.to_sparse())


In [14]:
product_treatment_matrix.to_dense()

tensor([[ 80.,  78.],
        [ 90., 128.],
        [ 45.,  52.],
        ...,
        [  0.,   1.],
        [  0.,   0.],
        [  0.,   0.]])

In [19]:
user_treatment_matrix.to_dense()[9,:]

tensor([24066., 24263.])

In [15]:
user_user_matrix = torch.sparse.mm(sparse_matrix_edge_index, sparse_matrix_edge_index.t())

In [20]:
user_user_treatment_matrix = torch.sparse.mm(user_user_matrix, new_feature.to_sparse())

### Training and Testing functions

In [8]:
from typing import Callable
import torch_geometric as pyg
from torch.optim import Optimizer

from typing import Callable
import torch_geometric as pyg
from torch.optim import Optimizer

 
def train(mask: torch.tensor,  #np.ndarray, 
          model:torch.nn.Module, 
          xu: torch.tensor, 
          xp: torch.tensor, 
          edge_index: torch.tensor, 
          treatment: torch.tensor, 
          outcome: torch.tensor,
          optimizer: Optimizer, 
          criterion: Callable[[torch.tensor, torch.tensor, torch.tensor, torch.tensor], torch.tensor] ) -> torch.tensor:
    """
    Trains the model for one epoch.
    """
    model.train()
    optimizer.zero_grad()  # Clear gradients.

    pred_t, pred_c, hidden_treatment, hidden_control = model(xu, xp, edge_index)
    
    
    pred_t = model(torch.cat[xu,1], xp, edge_index)
    pred_c = model(torch.cat[xu,0], xp, edge_index)
    loss = criterion(treatment[mask], pred_t[mask], pred_c[mask], outcome[mask])
    
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    return loss



def test(mask: torch.tensor,  #np.ndarray, 
          model:torch.nn.Module, 
          xu: torch.tensor, 
          xp: torch.tensor, 
          edge_index: torch.tensor, 
          treatment: torch.tensor, 
          outcome: torch.tensor,
          criterion: Callable[[torch.tensor, torch.tensor, torch.tensor, torch.tensor], torch.tensor] ) -> torch.tensor:
    """
    Tests the model. 
    """
    model.eval()
    pred_t, pred_c, hidden_treatment, hidden_control = model(xu, xp, edge_index)
    loss = criterion(treatment[mask], pred_t[mask], pred_c[mask], outcome[mask])
    return loss



In [13]:
edge_index_up_current

tensor([[     9,      9,      9,  ..., 189599, 191531, 199951],
        [180660, 180674, 180688,  ..., 180632, 180632, 180632]])

In [ ]:
from torch import nn
from torch_geometric.nn import SAGEConv


class BipartiteSAGE2mod_interf(torch.nn.Module):

    def __init__(self, nfeat:int, nproduct:int , hidden_channels:int , out_channels: int, num_layers:int, dropout_rate:float =0):
        
        super().__init__()
        self.convs = torch.nn.ModuleList()

        self.user_embed = nn.Linear(nfeat, hidden_channels )
        self.item_embed =  nn.Linear(nproduct, hidden_channels)

        self.convs = torch.nn.ModuleList()

        for _ in range(num_layers):
            self.convs.append(SAGEConv((-1,-1), hidden_channels))
            
        
        self.num_layers = num_layers

        self.hidden_common1 = nn.Linear(hidden_channels + num_layers*hidden_channels, hidden_channels)
        self.hidden_common2 = nn.Linear(hidden_channels, hidden_channels)

        self.hidden_control = nn.Linear(hidden_channels, int(hidden_channels/2))
        self.hidden_treatment = nn.Linear(hidden_channels, int(hidden_channels/2))

        self.out = nn.Linear( int(hidden_channels/2), out_channels)
        
        
        self.dropout = nn.Dropout(dropout_rate)
        
        self.activation = nn.ReLU()


    def forward(self, xu: torch.tensor, xp:torch.tensor, edge_index:torch._tensor):
        out = [] 
        xu = self.user_embed(xu)
        xp = self.item_embed(xp)

        out.append(xu)

        embeddings = torch.cat((xu,xp), dim=0) 
        
        for i in range(self.num_layers):
            embeddings = self.activation(self.convs[i](embeddings, edge_index))
            
            out.append(embeddings[:xu.shape[0]])            
        
        out = torch.cat( out, dim=1)
        
        hidden = self.dropout(self.activation(self.hidden_common1(out)))
        hidden = self.dropout(self.activation(self.hidden_common2(hidden)))        
        
        out = self.activation(self.out(hidden))
        
        return out

### Model definition

In [9]:
from torch import nn
from torch_geometric.nn import SAGEConv


class BipartiteSAGE2mod(torch.nn.Module):

    def __init__(self, nfeat:int, nproduct:int , hidden_channels:int , out_channels: int, num_layers:int, dropout_rate:float =0):
        
        super().__init__()
        self.convs = torch.nn.ModuleList()

        self.user_embed = nn.Linear(nfeat, hidden_channels )
        self.item_embed =  nn.Linear(nproduct, hidden_channels)

        self.convs = torch.nn.ModuleList()

        for _ in range(num_layers):
            self.convs.append(SAGEConv((-1,-1), hidden_channels))
            
        
        self.num_layers = num_layers

        self.hidden_common1 = nn.Linear(hidden_channels + num_layers*hidden_channels, hidden_channels)
        self.hidden_common2 = nn.Linear(hidden_channels, hidden_channels)

        self.hidden_control = nn.Linear(hidden_channels, int(hidden_channels/2))
        self.hidden_treatment = nn.Linear(hidden_channels, int(hidden_channels/2))

        self.out_control = nn.Linear( int(hidden_channels/2), out_channels)
        self.out_treatment = nn.Linear( int(hidden_channels/2), out_channels)

        
        self.dropout = nn.Dropout(dropout_rate)
        
        self.activation = nn.ReLU()


    def forward(self, xu: torch.tensor, xp:torch.tensor, edge_index:torch._tensor):
        out = [] 
        xu = self.user_embed(xu)
        xp = self.item_embed(xp)

        out.append(xu)

        embeddings = torch.cat((xu,xp), dim=0) 
        
        for i in range(self.num_layers):
            embeddings = self.activation(self.convs[i](embeddings, edge_index))
            
            out.append(embeddings[:xu.shape[0]])            
        
        out = torch.cat( out, dim=1)
        
        hidden = self.dropout(self.activation(self.hidden_common1(out)))
        hidden = self.dropout(self.activation(self.hidden_common2(hidden)))
        
        hidden_1t0 = self.dropout(self.activation(self.hidden_control(hidden)))
        hidden_1t1 = self.dropout(self.activation(self.hidden_treatment(hidden)))

        out_2t0 = self.activation(self.out_control(hidden_1t0))
        out_2t1 = self.activation(self.out_treatment(hidden_1t1))
        
        return out_2t1, out_2t0, hidden_1t1, hidden_1t0

### Training

In [25]:
import numpy as np
from torch.optim import Adam


def objective(trial):
    model_file = "../models/bipartite_sage"

    early_stopping = 0
    train_losses = []
    val_losses = []
    best_val_loss = np.inf
    print_per_epoch = 50
    
            
    hidden_channels = trial.suggest_int("hidden_channels", 16, 64)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    lr = trial.suggest_float("lr", 1e-3, 1e-1)
    batch_size = trial.suggest_int("batch_size", 32, 128)
    
     
    model = BipartiteSAGE2mod(xu.shape[1], xp.shape[1] , hidden_channels, out_channels, num_layers, dropout_rate).to(device)

    optimizer = Adam(model.parameters(), lr=lr, weight_decay = l2_reg)

    # init params
    out = model(xu, xp, edge_index_up_current)

    data_loader = torch.utils.data.DataLoader(train_indices,
                            batch_size=batch_size,
                            shuffle=True)

    for epoch in range(num_epochs):
        mean_val_loss = 0
        for train_batch_index in data_loader:
            train_loss = train(train_batch_index, model, xu, xp, edge_index_up_current, treatment, outcome, optimizer, criterion)
            val_loss = test(val_indices, model, xu, xp, edge_index_up_current, treatment, outcome, criterion)
            
            mean_val_loss += val_loss/len(data_loader)
            train_losses.append(float(train_loss.item())) 
            
        val_losses.append(float(mean_val_loss.item()))

        if mean_val_loss < best_val_loss:
            early_stopping=0
            best_val_loss = mean_val_loss
            torch.save(model, model_file + f"_{trial.number}.pt")
        else:
            early_stopping += 1
            if early_stopping > patience:
                print("early stopping..")
                break

        if epoch%print_per_epoch==0:
            print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.4f}, Val loss: {mean_val_loss:.4f}') #, Test: {test_acc:.4f}'
        
    return best_val_loss

In [26]:
import optuna
from optuna.trial import TrialState

study = optuna.create_study(direction="minimize")
study.optimize(objective,
               n_trials=100)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
best_model = study.best_trial

model_file = "../models/bipartite_sage" + f"_{best_model.number}.pt"

print("  Value: ", best_model.value)

print("  Params: ")
for key, value in best_model.params.items():
    print("    {}: {}".format(key, value))

[I 2024-07-09 10:21:30,336] A new study created in memory with name: no-name-e5bd9d3f-b5d0-49cc-b446-aa8f99553ddc


Epoch: 000, Train Loss: 216.4955, Val loss: 220.3088


[I 2024-07-09 10:22:18,398] Trial 0 finished with value: 220.30877685546875 and parameters: {'hidden_channels': 21, 'dropout_rate': 0.46293094056179407, 'num_layers': 3, 'lr': 0.017118077953755927}. Best is trial 0 with value: 220.30877685546875.


Epoch: 000, Train Loss: 220.4789, Val loss: 220.2345


[I 2024-07-09 10:23:02,334] Trial 1 finished with value: 220.21966552734375 and parameters: {'hidden_channels': 26, 'dropout_rate': 0.4785508637733481, 'num_layers': 1, 'lr': 0.0020037920926335795}. Best is trial 1 with value: 220.21966552734375.


Epoch: 000, Train Loss: 214.7193, Val loss: 191.3461


[I 2024-07-09 10:23:53,591] Trial 2 finished with value: 151.9998779296875 and parameters: {'hidden_channels': 27, 'dropout_rate': 0.29248676547613817, 'num_layers': 3, 'lr': 0.08066003013340728}. Best is trial 2 with value: 151.9998779296875.


Epoch: 000, Train Loss: 173.6843, Val loss: 501.2159


[I 2024-07-09 10:24:53,226] Trial 3 finished with value: 218.76510620117188 and parameters: {'hidden_channels': 52, 'dropout_rate': 0.20264813012825394, 'num_layers': 2, 'lr': 0.07578702306377659}. Best is trial 2 with value: 151.9998779296875.


Study statistics: 
  Number of finished trials:  4
  Number of pruned trials:  0
  Number of complete trials:  4
Best trial:
  Value:  151.9998779296875
  Params: 
    hidden_channels: 27
    dropout_rate: 0.29248676547613817
    num_layers: 3
    lr: 0.08066003013340728


### Testing

In [32]:
model = torch.load(model_file).to(device)
model.eval()

mask = test_indices
pred_t, pred_c, hidden_treatment, hidden_control = model(xu, xp, edge_index_up_current)

test_loss = criterion(treatment[mask], pred_t[mask], pred_c[mask], outcome[mask])

treatment_test = treatment[test_indices].detach().cpu().numpy()
outcome_test = outcome[test_indices].detach().cpu().numpy()
pred_t = pred_t.detach().cpu().numpy()
pred_c = pred_c.detach().cpu().numpy()


uplift = pred_t[test_indices] - pred_c[test_indices]
uplift = uplift.squeeze()

#mse = (uplift.mean() - (outcome_test[treatment_test==1].mean() - outcome_test[treatment_test==0].mean()))**2
print(f'mse {test_loss:.4f} with avg abs value {torch.mean(torch.abs(outcome[mask]))}')
up40 = uplift_score(uplift, treatment_test, outcome_test,0.4)
print(f'up40 {up40:.4f}')
up20 = uplift_score(uplift, treatment_test, outcome_test,0.2)
print(f'up20 {up20:.4f}')

mse 154.8895 with avg abs value 445.1011657714844
up40 6.7227
up20 7.5219


### Benchmarks

In [33]:
from causalml.inference.meta import  BaseXRegressor, BaseTRegressor
from causalml.propensity import ElasticNetPropensityModel
from xgboost import XGBRegressor
from causalml.inference.tree.causal.causaltree import CausalTreeRegressor

train_indices = np.hstack([train_indices, val_indices])
outcome = outcome.cpu().numpy()
xu = xu.cpu().numpy()
treatment = treatment.cpu().numpy()

Failed to import duecredit due to No module named 'duecredit'


In [34]:
learner = BaseTRegressor(learner = XGBRegressor())

learner.fit(X= xu[train_indices], y=outcome[train_indices], treatment= treatment[train_indices] )  
uplift=learner.predict(X = xu[train_indices], treatment = treatment[test_indices]).squeeze()

uplift=learner.predict(X = xu[test_indices], treatment= treatment[test_indices]).squeeze()

up40 = uplift_score(uplift, np.hstack([treatment[train_indices] ,treatment[test_indices]]), np.hstack([outcome[train_indices],outcome[test_indices]]), rate=0.4)
up20 = uplift_score(uplift, np.hstack([treatment[train_indices],treatment[test_indices]]), np.hstack([outcome[train_indices],outcome[test_indices]]), rate=0.2)

print(f'T-learner up40: {up40:.4f} , up20: {up20:.4f}')

T-learner up40: 0.5721 , up20: 1.7642


In [15]:
propensity_model = ElasticNetPropensityModel()

propensity_model.fit(X=xu[train_indices], y = treatment[train_indices])
p_train = propensity_model.predict(X=xu[train_indices])
p_test = propensity_model.predict(X=xu[test_indices])

learner = BaseXRegressor(learner = XGBRegressor())

learner.fit(X= xu[train_indices], y=outcome[train_indices], treatment= treatment[train_indices], p=p_train )  

uplift=learner.predict(X = xu[test_indices], treatment= treatment[test_indices], p=p_test).squeeze()

up40 = uplift_score(uplift, np.hstack([treatment[train_indices] ,treatment[test_indices]]), np.hstack([outcome[train_indices],outcome[test_indices]]), rate=0.4)
up20 = uplift_score(uplift, np.hstack([treatment[train_indices],treatment[test_indices]]), np.hstack([outcome[train_indices],outcome[test_indices]]), rate=0.2)

print(f'X-learner up40: {up40:.4f} , up20: {up20:.4f}')

The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge
The max_iter was reached which means the coef_ did not converge


X-learner up40: -1.6282 , up20: -2.2200


In [16]:
learner = CausalTreeRegressor(control_name="0")
X_train = np.hstack(( treatment[train_indices].reshape(-1, 1), xu[train_indices]))
X_test = np.hstack((treatment[test_indices].reshape(-1, 1),  xu[test_indices]))
learner.fit( X = X_train, treatment = treatment[train_indices].astype(str), y=outcome[train_indices])
uplift = learner.predict( X = X_test).squeeze()

up40 = uplift_score(uplift, np.hstack([treatment[train_indices] ,treatment[test_indices]]), np.hstack([outcome[train_indices],outcome[test_indices]]), rate=0.4)
up20 = uplift_score(uplift, np.hstack([treatment[train_indices],treatment[test_indices]]), np.hstack([outcome[train_indices],outcome[test_indices]]), rate=0.2)

print(f'Tree up40: {up40:.4f} , up20: {up20:.4f}')

Tree up40: -1.6412 , up20: 5.2740
